In [18]:
import pandas as pd
from requests import get
import re
from bs4 import BeautifulSoup
import numpy as np

In [2]:
df=pd.read_csv('batting_data.csv')
df

,player_id,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Opposition,Ground,Start,odi_number
0,440970,0,-,3,0,0,0.00,3,caught,2,Kenya,Sharjah,2 Oct 2013,ODI # 3417
1,440970,13*,-,19,0,0,68.42,5,not out,2,Kenya,Sharjah,4 Oct 2013,ODI # 3418
2,440970,38*,-,67,4,0,56.71,4,not out,2,Hong Kong,Kuala Lumpur,1 May 2014,ODI # 3487
3,440970,3,-,7,0,0,42.85,4,lbw,1,U.A.E.,Kuala Lumpur,2 May 2014,ODI # 3488
4,440970,15,21,22,2,0,68.18,4,caught,1,Zimbabwe,Bulawayo,18 Jul 2014,ODI # 3503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9326,793007,4*,4,2,1,0,200.00,11,not out,2,South Africa,Delhi,7 Oct 2023,ODI # 4661
9327,793007,DNB,-,-,-,-,-,-,-,1,Pakistan,Hyderabad,10 Oct 2023,ODI # 4665
9328,793007,0*,12,6,0,0,0.00,11,not out,1,Australia,Lucknow,16 Oct 2023,ODI # 4671
9329,793007,DNB,-,-,-,-,-,-,-,2,Netherlands,Lucknow,21 Oct 2023,ODI # 4676


In [5]:
df.value_counts

<bound method DataFrame.value_counts of       player_id  Runs Mins  BF 4s 6s      SR Pos Dismissal Inns    Opposition  \
0        440970     0    -   3  0  0    0.00   3    caught    2         Kenya   
1        440970   13*    -  19  0  0   68.42   5   not out    2         Kenya   
2        440970   38*    -  67  4  0   56.71   4   not out    2     Hong Kong   
3        440970     3    -   7  0  0   42.85   4       lbw    1        U.A.E.   
4        440970    15   21  22  2  0   68.18   4    caught    1      Zimbabwe   
...         ...   ...  ...  .. .. ..     ...  ..       ...  ...           ...   
9326     793007    4*    4   2  1  0  200.00  11   not out    2  South Africa   
9327     793007   DNB    -   -  -  -       -   -         -    1      Pakistan   
9328     793007    0*   12   6  0  0    0.00  11   not out    1     Australia   
9329     793007   DNB    -   -  -  -       -   -         -    2   Netherlands   
9330     793007  TDNB    -   -  -  -       -   -         -    -      

In [11]:

# Assuming df is your DataFrame
df = df[(df['Runs'] != 'DNB') & (df['Runs'] != 'TDNB') & (df['Mins'] != '-')]
df


,player_id,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Opposition,Ground,Start,odi_number
4,440970,15,21,22,2,0,68.18,4,caught,1,Zimbabwe,Bulawayo,18 Jul 2014,ODI # 3503
5,440970,11,25,18,0,0,61.11,4,caught,1,Zimbabwe,Bulawayo,20 Jul 2014,ODI # 3504
9,440970,72,147,110,6,0,65.45,4,caught,2,Bangladesh,Mirpur,25 Sep 2016,ODI # 3781
10,440970,14,40,29,1,0,48.27,4,lbw,2,Bangladesh,Mirpur,28 Sep 2016,ODI # 3783
11,440970,0,2,3,0,0,0.00,4,caught,2,Bangladesh,Mirpur,1 Oct 2016,ODI # 3786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9321,793007,4*,7,2,1,0,200.00,11,not out,2,New Zealand,Auckland,25 Mar 2023,ODI # 4548
9322,793007,1*,6,5,0,0,20.00,11,not out,1,Netherlands,Bulawayo,30 Jun 2023,ODI # 4606
9325,793007,0*,2,0,0,0,-,11,not out,1,Netherlands,Harare,9 Jul 2023,ODI # 4620
9326,793007,4*,4,2,1,0,200.00,11,not out,2,South Africa,Delhi,7 Oct 2023,ODI # 4661


In [13]:
df1 = df.reset_index(drop=True)
df1

,player_id,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Opposition,Ground,Start,odi_number
0,440970,15,21,22,2,0,68.18,4,caught,1,Zimbabwe,Bulawayo,18 Jul 2014,ODI # 3503
1,440970,11,25,18,0,0,61.11,4,caught,1,Zimbabwe,Bulawayo,20 Jul 2014,ODI # 3504
2,440970,72,147,110,6,0,65.45,4,caught,2,Bangladesh,Mirpur,25 Sep 2016,ODI # 3781
3,440970,14,40,29,1,0,48.27,4,lbw,2,Bangladesh,Mirpur,28 Sep 2016,ODI # 3783
4,440970,0,2,3,0,0,0.00,4,caught,2,Bangladesh,Mirpur,1 Oct 2016,ODI # 3786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6256,793007,4*,7,2,1,0,200.00,11,not out,2,New Zealand,Auckland,25 Mar 2023,ODI # 4548
6257,793007,1*,6,5,0,0,20.00,11,not out,1,Netherlands,Bulawayo,30 Jun 2023,ODI # 4606
6258,793007,0*,2,0,0,0,-,11,not out,1,Netherlands,Harare,9 Jul 2023,ODI # 4620
6259,793007,4*,4,2,1,0,200.00,11,not out,2,South Africa,Delhi,7 Oct 2023,ODI # 4661


In [14]:
df

,player_id,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Opposition,Ground,Start,odi_number
4,440970,15,21,22,2,0,68.18,4,caught,1,Zimbabwe,Bulawayo,18 Jul 2014,ODI # 3503
5,440970,11,25,18,0,0,61.11,4,caught,1,Zimbabwe,Bulawayo,20 Jul 2014,ODI # 3504
9,440970,72,147,110,6,0,65.45,4,caught,2,Bangladesh,Mirpur,25 Sep 2016,ODI # 3781
10,440970,14,40,29,1,0,48.27,4,lbw,2,Bangladesh,Mirpur,28 Sep 2016,ODI # 3783
11,440970,0,2,3,0,0,0.00,4,caught,2,Bangladesh,Mirpur,1 Oct 2016,ODI # 3786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9321,793007,4*,7,2,1,0,200.00,11,not out,2,New Zealand,Auckland,25 Mar 2023,ODI # 4548
9322,793007,1*,6,5,0,0,20.00,11,not out,1,Netherlands,Bulawayo,30 Jun 2023,ODI # 4606
9325,793007,0*,2,0,0,0,-,11,not out,1,Netherlands,Harare,9 Jul 2023,ODI # 4620
9326,793007,4*,4,2,1,0,200.00,11,not out,2,South Africa,Delhi,7 Oct 2023,ODI # 4661


In [16]:
df1['Runs'].value_counts


<bound method IndexOpsMixin.value_counts of 0       15
1       11
2       72
3       14
4        0
        ..
6256    4*
6257    1*
6258    0*
6259    4*
6260    0*
Name: Runs, Length: 6261, dtype: object>

In [19]:
df1

,player_id,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Opposition,Ground,Start,odi_number
0,440970,15,21,22,2,0,68.18,4,caught,1,Zimbabwe,Bulawayo,18 Jul 2014,ODI # 3503
1,440970,11,25,18,0,0,61.11,4,caught,1,Zimbabwe,Bulawayo,20 Jul 2014,ODI # 3504
2,440970,72,147,110,6,0,65.45,4,caught,2,Bangladesh,Mirpur,25 Sep 2016,ODI # 3781
3,440970,14,40,29,1,0,48.27,4,lbw,2,Bangladesh,Mirpur,28 Sep 2016,ODI # 3783
4,440970,0,2,3,0,0,0.00,4,caught,2,Bangladesh,Mirpur,1 Oct 2016,ODI # 3786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6256,793007,4*,7,2,1,0,200.00,11,not out,2,New Zealand,Auckland,25 Mar 2023,ODI # 4548
6257,793007,1*,6,5,0,0,20.00,11,not out,1,Netherlands,Bulawayo,30 Jun 2023,ODI # 4606
6258,793007,0*,2,0,0,0,-,11,not out,1,Netherlands,Harare,9 Jul 2023,ODI # 4620
6259,793007,4*,4,2,1,0,200.00,11,not out,2,South Africa,Delhi,7 Oct 2023,ODI # 4661


In [20]:
from sklearn.preprocessing import LabelEncoder
le_o=LabelEncoder()
le_g=LabelEncoder()
le_d=LabelEncoder()
df1['Dismissal_n']=le_d.fit_transform(df1['Dismissal'])
df1['Opposition_n']=le_o.fit_transform(df1['Opposition'])
df1['Ground_n']=le_g.fit_transform(df1['Ground'])
df1

,player_id,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Opposition,Ground,Start,odi_number,Dismissal_n,Opposition_n,Ground_n
0,440970,15,21,22,2,0,68.18,4,caught,1,Zimbabwe,Bulawayo,18 Jul 2014,ODI # 3503,1,20,18
1,440970,11,25,18,0,0,61.11,4,caught,1,Zimbabwe,Bulawayo,20 Jul 2014,ODI # 3504,1,20,18
2,440970,72,147,110,6,0,65.45,4,caught,2,Bangladesh,Mirpur,25 Sep 2016,ODI # 3781,1,2,80
3,440970,14,40,29,1,0,48.27,4,lbw,2,Bangladesh,Mirpur,28 Sep 2016,ODI # 3783,3,2,80
4,440970,0,2,3,0,0,0.00,4,caught,2,Bangladesh,Mirpur,1 Oct 2016,ODI # 3786,1,2,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6256,793007,4*,7,2,1,0,200.00,11,not out,2,New Zealand,Auckland,25 Mar 2023,ODI # 4548,4,11,6
6257,793007,1*,6,5,0,0,20.00,11,not out,1,Netherlands,Bulawayo,30 Jun 2023,ODI # 4606,4,10,18
6258,793007,0*,2,0,0,0,-,11,not out,1,Netherlands,Harare,9 Jul 2023,ODI # 4620,4,10,58
6259,793007,4*,4,2,1,0,200.00,11,not out,2,South Africa,Delhi,7 Oct 2023,ODI # 4661,4,15,36


In [21]:
df2=df1.drop(['Dismissal','Opposition','Ground'],axis='columns')
df2

,player_id,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Start,odi_number,Dismissal_n,Opposition_n,Ground_n
0,440970,15,21,22,2,0,68.18,4,1,18 Jul 2014,ODI # 3503,1,20,18
1,440970,11,25,18,0,0,61.11,4,1,20 Jul 2014,ODI # 3504,1,20,18
2,440970,72,147,110,6,0,65.45,4,2,25 Sep 2016,ODI # 3781,1,2,80
3,440970,14,40,29,1,0,48.27,4,2,28 Sep 2016,ODI # 3783,3,2,80
4,440970,0,2,3,0,0,0.00,4,2,1 Oct 2016,ODI # 3786,1,2,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6256,793007,4*,7,2,1,0,200.00,11,2,25 Mar 2023,ODI # 4548,4,11,6
6257,793007,1*,6,5,0,0,20.00,11,1,30 Jun 2023,ODI # 4606,4,10,18
6258,793007,0*,2,0,0,0,-,11,1,9 Jul 2023,ODI # 4620,4,10,58
6259,793007,4*,4,2,1,0,200.00,11,2,7 Oct 2023,ODI # 4661,4,15,36


In [24]:
df2.describe

<bound method NDFrame.describe of       player_id Runs Mins   BF 4s 6s      SR Pos Inns        Start  \
0        440970   15   21   22  2  0   68.18   4    1  18 Jul 2014   
1        440970   11   25   18  0  0   61.11   4    1  20 Jul 2014   
2        440970   72  147  110  6  0   65.45   4    2  25 Sep 2016   
3        440970   14   40   29  1  0   48.27   4    2  28 Sep 2016   
4        440970    0    2    3  0  0    0.00   4    2   1 Oct 2016   
...         ...  ...  ...  ... .. ..     ...  ..  ...          ...   
6256     793007   4*    7    2  1  0  200.00  11    2  25 Mar 2023   
6257     793007   1*    6    5  0  0   20.00  11    1  30 Jun 2023   
6258     793007   0*    2    0  0  0       -  11    1   9 Jul 2023   
6259     793007   4*    4    2  1  0  200.00  11    2   7 Oct 2023   
6260     793007   0*   12    6  0  0    0.00  11    1  16 Oct 2023   

      odi_number  Dismissal_n  Opposition_n  Ground_n  
0     ODI # 3503            1            20        18  
1     ODI # 3

In [34]:
df2['Runs'].unique()

array(['15', '11', '72', '14', '0', '21', '2*', '12', '23', '54', '2',
       '34*', '47', '18', '4', '59', '8', '76', '82', '10', '73', '28',
       '5', '88', '38', '57', '9*', '22', '15*', '13', '51', '80', '48*',
       '127', '31', '7', '106*', '17', '53', '68', '145', '6', '151', '1',
       '65', '19', '106', '162', '98', '41*', '100', '75', '87', '50',
       '35', '34', '9', '71', '36', '108*', '69', '16', '29', '32', '33',
       '62', '43', '46', '24', '3', '103*', '70', '48', '52', '94', '58',
       '55', '45', '77*', '56', '12*', '25', '26', '67', '63*', '17*',
       '42', '60*', '1*', '3*', '77', '7*', '8*', '6*', '37', '44', '116',
       '30', '49', '41', '92', '27', '20', '27*', '25*', '86', '19*',
       '16*', '13*', '39*', '4*', '18*', '64', '10*', '0*', '11*', '23*',
       '21*', '117', '60', '73*', '55*', '38*', '85', '45*', '26*', '42*',
       '99', '40*', '89*', '20*', '5*', '40', '39', '128', '71*', '101',
       '89', '70*', '152', '51*', '108', '80*', '12

In [35]:
df2['Runs']=df2['Runs'].str.replace('*','')
df2

,player_id,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Start,odi_number,Dismissal_n,Opposition_n,Ground_n
0,440970,15,21,22,2,0,68.18,4,1,18 Jul 2014,ODI # 3503,1,20,18
1,440970,11,25,18,0,0,61.11,4,1,20 Jul 2014,ODI # 3504,1,20,18
2,440970,72,147,110,6,0,65.45,4,2,25 Sep 2016,ODI # 3781,1,2,80
3,440970,14,40,29,1,0,48.27,4,2,28 Sep 2016,ODI # 3783,3,2,80
4,440970,0,2,3,0,0,0.00,4,2,1 Oct 2016,ODI # 3786,1,2,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6256,793007,4,7,2,1,0,200.00,11,2,25 Mar 2023,ODI # 4548,4,11,6
6257,793007,1,6,5,0,0,20.00,11,1,30 Jun 2023,ODI # 4606,4,10,18
6258,793007,0,2,0,0,0,-,11,1,9 Jul 2023,ODI # 4620,4,10,58
6259,793007,4,4,2,1,0,200.00,11,2,7 Oct 2023,ODI # 4661,4,15,36


In [36]:
df2['Runs'].unique()

array(['15', '11', '72', '14', '0', '21', '2', '12', '23', '54', '34',
       '47', '18', '4', '59', '8', '76', '82', '10', '73', '28', '5',
       '88', '38', '57', '9', '22', '13', '51', '80', '48', '127', '31',
       '7', '106', '17', '53', '68', '145', '6', '151', '1', '65', '19',
       '162', '98', '41', '100', '75', '87', '50', '35', '71', '36',
       '108', '69', '16', '29', '32', '33', '62', '43', '46', '24', '3',
       '103', '70', '52', '94', '58', '55', '45', '77', '56', '25', '26',
       '67', '63', '42', '60', '37', '44', '116', '30', '49', '92', '27',
       '20', '86', '39', '64', '117', '85', '99', '40', '89', '128',
       '101', '152', '124', '102', '79', '81', '66', '96', '121', '93',
       '95', '78', '146', '74', '163', '178', '84', '122', '109', '173',
       '119', '156', '130', '179', '107', '166', '104', '105', '114',
       '61', '136', '90', '110', '83', '125', '111', '112', '129', '91',
       '131', '138', '134', '118', '140', '113', '133', '182', '14